## 連動 Google Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

!mkdir -p /drive
#umount /drive
!mount --bind /content/drive/My\ Drive /drive
!mkdir -p /drive/ngrok-ssh
!mkdir -p ~/.ssh

Mounted at /content/drive
shell-init: error retrieving current directory: getcwd: cannot access parent directories: No such file or directory
mkdir: cannot stat ‘/drive’: Transport endpoint is not connected
shell-init: error retrieving current directory: getcwd: cannot access parent directories: No such file or directory
shell-init: error retrieving current directory: getcwd: cannot access parent directories: No such file or directory
shell-init: error retrieving current directory: getcwd: cannot access parent directories: No such file or directory


## 下載 ngrok 檔案

In [ ]:
!mkdir -p /drive/ngrok-ssh
%cd /drive/ngrok-ssh
!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip -O ngrok-stable-linux-amd64.zip
!unzip -u ngrok-stable-linux-amd64.zip
!cp /drive/ngrok-ssh/ngrok /ngrok
!chmod +x /ngrok

shell-init: error retrieving current directory: getcwd: cannot access parent directories: No such file or directory
/drive/ngrok-ssh
--2023-12-30 10:21:20--  https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
Resolving bin.equinox.io (bin.equinox.io)... 52.202.168.65, 54.161.241.46, 54.237.133.81, ...
Connecting to bin.equinox.io (bin.equinox.io)|52.202.168.65|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 13921656 (13M) [application/octet-stream]
Saving to: ‘ngrok-stable-linux-amd64.zip’

ngrok-stable-linux- 100%[===================>]  13.28M  18.3MB/s    in 0.7s    

2023-12-30 10:21:21 (18.3 MB/s) - ‘ngrok-stable-linux-amd64.zip’ saved [13921656/13921656]

Archive:  ngrok-stable-linux-amd64.zip


## 輸入 ngrok token

In [ ]:
!/ngrok authtoken "authtoken"

Authtoken saved to configuration file: /root/.ngrok2/ngrok.yml


## 安裝 flask_ngrok 函式庫

In [ ]:
!pip install flask_ngrok

## 安裝 line-bot-sdk 函式庫

In [ ]:
!pip install line-bot-sdk

## LINE BOT 程式

In [ ]:
! pip install openai

In [ ]:
from flask_ngrok import run_with_ngrok
from flask import Flask, request
from linebot import LineBotApi, WebhookHandler
from linebot.models import TextSendMessage
import json
from openai import OpenAI

access_token = "access_token"
channel_secret = "channel_secret"
api_key = "api_key"

app = Flask(__name__)

@app.route("/", methods=['POST'])
def linebot():
    try:
        body = request.get_data(as_text=True)
        json_data = json.loads(body)
        print(json_data)

        # Check if the required fields exist in the JSON data
        events = json_data.get('events', [])
        if not events:
            raise ValueError("No 'events' field found in JSON data")

        tk = events[0].get('replyToken', '')
        msg = events[0].get('message', {}).get('text', '')

        if not tk or not msg:
            raise ValueError("Missing 'replyToken' or 'message.text' in JSON data")

        line_bot_api = LineBotApi(access_token)
        handler = WebhookHandler(channel_secret)
        signature = request.headers['X-Line-Signature']
        handler.handle(body, signature)


        openai.api_key = api_key

        completion = openai.chat.completions.create(
          model="gpt-3.5-turbo",
          messages=[
            {
            "role": "user",
            "content": msg[0:],
            },
          ],
        )
        response = completion.choices[0].message.content

        text_message = TextSendMessage(text=response)
        line_bot_api.reply_message(tk, text_message)

    except Exception as e:
        print(f'Error: {e}')

    return 'OK'

if __name__ == "__main__":
    run_with_ngrok(app)
    app.run()


 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit


 * Running on http://3cfb-34-106-31-173.ngrok-free.app
 * Traffic stats available on http://127.0.0.1:4040


INFO:werkzeug:127.0.0.1 - - [30/Dec/2023 12:46:52] "POST / HTTP/1.1" 200 -


{'destination': 'Uba0e65b06e838e564326ec1de43585fc', 'events': []}
Error: No 'events' field found in JSON data
{'destination': 'Uba0e65b06e838e564326ec1de43585fc', 'events': [{'type': 'message', 'message': {'type': 'text', 'id': '488400047777251714', 'quoteToken': '1JJXgKzfqwBAYjxlYbOucs0eVv14YlY_qGUNSMRzPsth3aROyRoGX-__LYC2a1ye1YLt7Ojf3eUYm5X5omb6WHY0QysehBf81v_ieSqmE3TuNKDL7UvTlhr3PDUX7gN_LYzUQJNn3B7P5Xl8oiAwiQ', 'text': 'hi'}, 'webhookEventId': '01HJXC6TEHXGMNRJEJ4TEV7KVC', 'deliveryContext': {'isRedelivery': False}, 'timestamp': 1703940417735, 'source': {'type': 'user', 'userId': 'U5d14e63c2c90619b7dc2ae0dee99cf6f'}, 'replyToken': 'd8aae80acf4f4f11a5feb5ee6c67a5e9', 'mode': 'active'}]}


...

INFO:werkzeug:127.0.0.1 - - [30/Dec/2023 12:57:51] "POST / HTTP/1.1" 200 -


{'destination': 'Uba0e65b06e838e564326ec1de43585fc', 'events': [{'type': 'message', 'message': {'type': 'text', 'id': '488401150962565479', 'quoteToken': 'papVUrcVqVEZ5wa50Yq7nKHQ--_xuMwb5HIMr9lS3scXXJtgPiRPEYeFnkeqQ6-LIUid9KwTmdMRHd8dCfc95rQVkWXdjrYjuiud3BdyqAGqHxBSDOjEHNNEwrlGTTR-PdUkMHniuhtN1Fc1Rmh5Fg', 'text': '請列出時下年輕人最常使用的十大社群網站，並評比智障分數'}, 'webhookEventId': '01HJXCTWMQDGYQNYHG024M6J3R', 'deliveryContext': {'isRedelivery': False}, 'timestamp': 1703941075298, 'source': {'type': 'user', 'userId': 'U5d14e63c2c90619b7dc2ae0dee99cf6f'}, 'replyToken': '74513978497549d0ab516a7a6d77a4d8', 'mode': 'active'}]}


INFO:werkzeug:127.0.0.1 - - [30/Dec/2023 12:58:19] "POST / HTTP/1.1" 200 -


{'destination': 'Uba0e65b06e838e564326ec1de43585fc', 'events': [{'type': 'message', 'message': {'type': 'text', 'id': '488401640287109347', 'quoteToken': '35zhiAkLg0D2XeRRLQmpbHJcQyT12plM5mQqgKHmiTnapwqRFRToXmjEkkG7_phKJZkLqs3zCxsRfZoIzbzbGlF2iSTosINtnSsqCehKEumRgUYz01N4b1iN_dedWfslfGnWL2FAoHFQOip7dX8v7Q', 'text': '請從1開始往下數'}, 'webhookEventId': '01HJXD3S4QY5MKZ7E792VYJYA1', 'deliveryContext': {'isRedelivery': False}, 'timestamp': 1703941366799, 'source': {'type': 'user', 'userId': 'U5d14e63c2c90619b7dc2ae0dee99cf6f'}, 'replyToken': '3a9cdeecaae94cccb94ebe2b21909fde', 'mode': 'active'}]}


INFO:werkzeug:127.0.0.1 - - [30/Dec/2023 13:02:55] "POST / HTTP/1.1" 200 -


## 放到Google Cloud Functions用的

In [ ]:
from linebot import LineBotApi, WebhookHandler
from linebot.models import TextSendMessage
import json
import openai

access_token = "access_token"
channel_secret = "channel_secret"
line_bot_api = LineBotApi(access_token)


def linebot(request):
    body = request.get_data(as_text=True)
    json_data = json.loads(body)
    try:
        handler = WebhookHandler(channel_secret)
        signature = request.headers['X-Line-Signature']
        handler.handle(body, signature)
        tk = json_data['events'][0]['replyToken']
        msg = json_data['events'][0]['message']['text']

        openai.api_key = "api_key"

        completion = openai.chat.completions.create(
          model="gpt-3.5-turbo",
          messages=[
            {
            "role": "user",
            "content": msg[0:],
            },
          ],
        )
        response = completion.choices[0].message.content

        text_message = TextSendMessage(text=response)
        line_bot_api.reply_message(tk, text_message)
    except:
        print('error')
    return 'OK'


<ipython-input-40-8d70c6ebdd6c>:8: LineBotSdkDeprecatedIn30: Call to deprecated class LineBotApi. (Use v3 class; linebot.v3.<feature>. See https://github.com/line/line-bot-sdk-python/blob/master/README.rst for more details.) -- Deprecated since version 3.0.0.
  line_bot_api = LineBotApi(access_token)
